In [1]:
################### import packages ##########################################
import pandas as pd
import re
import numpy as np
import os
import copy 
import fnmatch
%load_ext line_profiler
pd.set_option('display.max_columns', None)

##############################################################################
####### choose variables to look for from codebooks ###########################
##############################################################################

# make sure these are all lowercase!!! #####################

# for HRS core files, 2002 - present
#SELECT VARIABLES OF INTEREST

variables_to_look_for = { 'a500':'rmonth_survey'
                         , 'a501':'ryear_survey'
                         , 'x026m':'state_live'
                         , 'x060_r':'gender'
                         , 'x067_r':'year_born'
                         , 'a019':'age'
                         , 'a028':'live_nursing_facility'
                         , 'a066':'state_nursing_facility'
                         , 'a068m':'year_born'
                         , 'b002':'born_us' #1 yes, #5 no
                         , 'b005m':'country_born' #1 yes, #5 no
                         , 'b003m':'state_born'
                         , 'b014':'education'
                         , 'b017m':'highest_degree'
                         , 'b028':'race_latino_hispanic'
                         , 'b031a':'race'
                         , 'q020':'income'
                         , 'b033':'num_children'
                        , 'b050':'religion'
                         , 'marital':'marital_status'
                         , 'b058':'divorce_widow_status'
                         ,'c115':'exercise'
                         , 'c001':'health_status' #1 excellent, 5 poor
                         , 'c002':'health_status_compared_to_prev'
                          , 'c003':'health_status_better_compared_to_prev_degree'
, 'c004':'health_status_worse_compared_to_prev_degree'
, 'c005':'high_blood_pressure'
, 'c009':'high_blood_pressure_compared_to_prev'
, 'c010':'diabetes'
, 'c016':'diabetes_compared_to_prev'
, 'c018':'cancer'
, 'c023':'cancer_compared_to_prev'
, 'c030':'lung_disease'
, 'c031':'lung_disease_compared_to_prev'
, 'c036':'heart_condition'
, 'c039':'heart_condition_compared_to_prev'
, 'c053':'stroke'
, 'c055':'heart_condition_compared_to_prev'
, 'c065':'psychiatric_emotional_problem'
, 'c066':'psychiatric_emotional_problem_compared_to_prev'
, 'c070':'arthritis'
, 'c071':'arthritis_compared_to_prev'
, 'c104':'pain'
, 'c105':'pain_degree'
, 'c116':'smoke'
, 'c117':'still_smoke'
, 'c128':'alcohol'
, 'c129':'num_drinks_days_per_week'
, 'c130':'num_drinks_per_day'
, 'c144':'shortness_breath'
, 'c148':'fatigue'
, 'c149':'cough'
, 'c150':'depressed'
, 'c154':'tired'
, 'c155':'appetite_loss'
, 'c156':'appetite_increase'
, 'd101':'memory' #1 good, 5 poor
, 'd110':'depressed' #1 yes, #5 no
, 'e012':'children_nearby'
  , 'e118':'dependents'
            , 'e119':'num_dependents'             
                         , 'h002':'type_house'
                         , 'h004':'own_or_rent'
                         , 'h020':'property_value'
                         , 'h070':'year_property_acquired'
                         , 'j005m1':'employment_status'
                         , 'j018':'year_retired'
                       , 'p004':'financial_expectation'
                       , 'p007':'leave_inheritance'
                       , 'q481':'debt_amount'
                       , '005':'vocab_level'
                       , '034':'friendliness'
                       , '035':'attentiveness'
                       , '037':'cooperation'
                       , '038':'tiredness'
                       , '040':'memory'
                       , '041':'hearing'
                         }


# for HRS core files, pre 2002
# 2000 has g's, 1998 has f's, 1996 has e's, 1994 has w's, 1992 has v's
variables_to_look_for_pre_2002 = {
      'g775': 'rmonth_survey'
    , 'f704': 'rmonth_survey'
    , 'e391': 'rmonth_survey'
    , 'w120': 'rmonth_survey'
    , 'v128': 'rmonth_survey'
    , 'g774': 'ryear_survey'
    , 'f703': 'ryear_survey'
    , 'e393': 'ryear_survey'
    , 'w122': 'ryear_survey'
    , 'v127': 'ryear_survey'
    , 'g4996': 'rpr_lose_job'
    , 'f4583': 'rpr_lose_job'
    , 'e3788': 'rpr_lose_job'
    , 'w5801': 'rpr_lose_job' 
    , 'v3205': 'rpr_lose_job'    
}

# for CAMS core files
CAMS_variables = { 'b1': 'rauto_pur_f' 
                , 'b2': 'rrefrig_pur_f' 
                , 'b3': 'rwasherdrier_pur_f'
                , 'b4': 'rdishwasher_pur_f'
                , 'b5': 'rtv_pur_f'}

# for HRS Rand file, choose patterns
#* is a wildcard for wave, so gets it for all waves
hrs_patterns = ['r*mstat', 'r*agey_b', 'h*itot']

# for HRS Rand file, choose patterns
#cams_patterns = ['h*ctots', 'h*cdurs', 'h*cndur', 'h*ctotc']
cams_patterns = ['h*cctots', 'h*ccdurs', 'h*ccdur', 'h*ctot']

In [ ]:
# this is an extension of the sas_read_year function to work across multiple years
# for all years specified, gets all variables specified from the HRS core files

# input types:
# years :  list
# directory : string
# variables_to_look_for : dictionary
# variables_to_look_for_pre_2002 : dictionary

def read_all_years(years, directory, variables_to_look_for, variables_to_look_for_pre_2002):
    frames = []
    for i in range(len(years)):
        yr = years[i]
        print('reading files for survey year: ' + str(yr))
        temp_df = sas_read_year(directory, variables_to_look_for, yr, variables_to_look_for_pre_2002)
        temp_df = temp_df.reset_index(drop=True)  # Ensure unique index values
        frames.append(temp_df)
        if yr == 1992:
            temp_df = temp_dftgfhfghfghfghfghfghjimport rtoirmtrdex(drop=True) #1992 may have a duplicate problem
        print('\n')
    return frames
    final_frame = pd.concat(frames, ignore_index=True).drop_duplicates().reset_index(drop=True)
    return final_frame

In [12]:
var_dict = {
      1992: ''
    , 1994: ''
    , 1996: ''
    , 1998: ''
    , 2000: ''
    , 2002: 'h'
    , 2004: 'j' # they skip i for some reason
    , 2006: 'k'
    , 2008: 'l'
    , 2010: 'm'
    , 2012: 'n'
    , 2014: 'o'
    , 2016: 'p' 
    , 2018: 'q'
    , 2020: 'r'
}

def sas_read_year(directory, variables_to_look_for, survey_year, variables_to_look_for_pre_2002):
    # HRS and CAMS assignment
    id_variables = ['hhid', 'pn']
    
    # HRS assignments
    if survey_year < 2001:
        variables_to_look_for = variables_to_look_for_pre_2002  
    # get directories given year and survey type
    directory_sas = directory + 'h' + str(survey_year)[2:4] + 'sas'
    directory_da = directory + 'h' + str(survey_year)[2:4] + 'da'
    # get survey specific variables
    if variables_to_look_for_pre_2002 != 'CAMS':
        survey_specific_variables = [var_dict[survey_year] + x for x in variables_to_look_for]

    # CAMS assignments
    if variables_to_look_for_pre_2002 == 'CAMS':
        # survey specific variables
        survey_specific_variables = [x + '_' + str(survey_year)[2:4] for x in variables_to_look_for]
        # CAMS directories
        # 2001 has a different directory name
        if survey_year == 2001:
            ext = 'cams' + str(survey_year)[2:4]
            ext_da = ext
        # 2009 has a different directory structure
        elif survey_year == 2009:
            ext = 'cams' + str(survey_year) + '/sas'
            ext_da = 'cams' + str(survey_year) + '/data'
        else:
            ext = 'cams' + str(survey_year)
            ext_da = ext
        directory_sas = directory + ext
        directory_da = directory + ext_da

    # get list of paths: pre 1996 and some cams dont have R in them
    if survey_year < 1996:
        path_list = sorted(list(set([directory_sas + '/' +  x.split('.')[0] 
                                for x in os.listdir(directory_sas) 
                                    if 'sas' in x.lower()])))
    else:
        path_list = sorted(list(set([directory_sas + '/' +  x.split('.')[0] 
                                for x in os.listdir(directory_sas) 
                                    if ('sas' in x.lower())
                                    and (
                                          ('r' in x[len(x.split('.')[0])-1].lower()) #respondent
                                        | ('h' in x[len(x.split('.')[0])-1].lower()) #household
                                        )
                                     
                            ]
                           )
                       )
                  )
        
    # from paths, get paths leading to files which have our desired variables
    # earlier than 2000, need to read all surveys because naming conventions and variables are unrelated
    # after 2000, only reads files with desired variables in them to save a lot of time
    path_short_list = path_list
    #print(path_short_list)
    '''
    if (survey_year > 2000) and (variables_to_look_for_pre_2002 != 'CAMS'):
        surveys = [x[0] for x in variables_to_look_for] + [x[0:2] for x in variables_to_look_for if 'lb' in x]
        path_short_list = [x for x in path_list if (('r' in x.split('/')[len(x.split('/'))-1].lower()  # get respondent
                                                        and any(y == x.split('/')[len(x.split('/'))-1].lower()[3:len(x.split('/')[len(x.split('/'))-1]) - 2] 
                                                                    for y in surveys) # get specific file
                                                    )  
                                                   ) 
                            or (('h' in x.split('/')[len(x.split('/'))-1].lower())  # get household
                                # take all surveys we are looking for, only keep if HH survey (this is the last list comprehension below)
                                and any(y == x.split('/')[len(x.split('/'))-1].lower()[3:len(x.split('/')[len(x.split('/'))-1]) - 2] 
                                                                    for y in [w for w in surveys if w in ['h', 'q', 'r', 'u']])
                                )
                           ]
    #
    (path_short_list)
    '''
    # rename variables to names chosen in input dictionary
    rename_dict = copy.deepcopy(variables_to_look_for)
    vars_list = [x for x in variables_to_look_for]
    for x in range(len(survey_specific_variables)):
        rename_dict[survey_specific_variables[x].upper()] = rename_dict.pop(vars_list[x])  
    print('looking for variables: ' + str(survey_specific_variables))
    
    # get proper file extensions
    # this maybe can get coded away if handle all the info in path and path short list better
    for i in range(len(path_short_list)):
        path = path_short_list[i]
        path_ind = len(path.split('/')) - 1
        if survey_year < 2001:
            ext_surv_dct = '/' + path.split('/')[path_ind] +  '.SAS' #in the year 2000, files are .SAS and not .sas
            ext_surv_da = '/' + path.split('/')[path_ind] + '.DA'
        else:
            ext_surv_dct = '/' + path.split('/')[path_ind] +  '.sas'
            ext_surv_da = '/' + path.split('/')[path_ind] + '.da'
        dct_file = directory_sas + ext_surv_dct
        filename = directory_da + ext_surv_da
        print(filename)
        # read the files
        temp_df = read_sas_fwf(dct_file, filename, survey_specific_variables)
        temp_df = temp_df.rename(columns = rename_dict)
        if i == 0:
            year_frame = temp_df
        else:
            try:
                year_frame = year_frame.merge(temp_df, on = ["HHID", "PN"], how = "outer")
            except:
                try:
                    ##### MERGE HOUSEHOLD DATA ON TO RESPONDENT LEVEL, GET DATASET UNIQUE AT THE HHID LEVEL ###############
                    # Lowest number is for the first person in the dataset with that HHID, so most likely oldest, or head of household(?)
                    ### We want the lowest QPN_FIN number for each HHID
                    ### If no QPN_FIN number for a HHID, select lowest QPN_FAM
                    ### IF no QPN_FIN or QPN_FAM, we want the lowest QPN_CS number
                    #temp_df = temp_df.sort_values(['HHID', 'PN_CS', 'PN_FIN', 'PN_FAM']).reset_index(drop = True)

                    # Keep top row for each HHID
                    #temp_df = temp_df.drop_duplicates(subset=['HHID'], keep = 'first')
                    
                    # THEN MERGE ONCE ROWS ARE UNIQUE TO HHID ###########
                    year_frame = year_frame.merge(temp_df, on = ["HHID"], how = "outer") # for households
                    print('household survey')
                except:
                    print('this is a pre-1996 household survey, so exclude') # pre-1996 some surveys wont have a PN

    # make the wave variable, 1992 was wave 1
    year_frame['wave'] = (survey_year - 1990) / 2
    return year_frame

In [13]:
def read_sas_fwf(dct_file, filename, survey_specific_variables):
    # global id variables
    id_vars = ['hhid', 'pn']

    # parse the .sas file for relevant lines
    colons = []
    with open(dct_file) as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if 'INPUT' in lines[i]:
            cols_start = i
        if 'LABEL' in lines[i]:
            names_start = i
        if ';' in lines[i]:
            colons.append(i)
    cols_end = [x for x in colons if x > cols_start and x < names_start][0]
    names_end = [x for x in colons if x > names_start][0]
    dictionary = lines[cols_start + 1: cols_end]
    
    # parse the relevant lines into outputs necessary for pd.read_fwf
    colspecs = []
    type_dict = {}
    names = []
    
    # can maybe make this faster by using pandas functions. unnecessary though
    for i in dictionary:
        
        # these comprehensions standardize the differences in sas files over the years
        temp = i.lstrip().split(' ')
        temp = [x.replace('\n', '').split('-') for x in temp]
        temp = [item for sublist in temp for item in sublist] # flatten the list
        temp = [i for i in temp if i]                         # removes empty strings
        
        # only read a variable if it matches survey specific variables
        if any([temp[0].lower() == y for y in id_vars + survey_specific_variables]):
            names.append(temp[0])
            if temp[1] == '$':
                var = str
                colspecs.append([int(temp[2]) - 1, int(temp[3])])
            else:
                var = float
                colspecs.append([int(temp[1]) - 1, int(temp[2])])
            type_dict[temp[0]] = var         
    return pd.read_fwf(filename, colspecs=colspecs, names = names, converters = type_dict)

In [14]:
cwd = os.getcwd()
directory = cwd + '/data/HRS/'
#years = [2002]
years = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016
        ]
output = read_all_years(years, directory, variables_to_look_for, variables_to_look_for_pre_2002)

reading files for survey year: 2002
looking for variables: ['ha500', 'ha501', 'hx026m', 'hx060_r', 'hx067_r', 'ha019', 'ha028', 'ha066', 'ha068m', 'hb002', 'hb005m', 'hb003m', 'hb014', 'hb017m', 'hb028', 'hb031a', 'hq020', 'hb033', 'hb050', 'hmarital', 'hb058', 'hc115', 'hc001', 'hc002', 'hc003', 'hc004', 'hc005', 'hc009', 'hc010', 'hc016', 'hc018', 'hc023', 'hc030', 'hc031', 'hc036', 'hc039', 'hc053', 'hc055', 'hc065', 'hc066', 'hc070', 'hc071', 'hc104', 'hc105', 'hc116', 'hc117', 'hc128', 'hc129', 'hc130', 'hc144', 'hc148', 'hc149', 'hc150', 'hc154', 'hc155', 'hc156', 'hd101', 'hd110', 'he012', 'he118', 'he119', 'hh002', 'hh004', 'hh020', 'hh070', 'hj005m1', 'hj018', 'hp004', 'hp007', 'hq481', 'h005', 'h034', 'h035', 'h037', 'h038', 'h040', 'h041']
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS/h02da/H02A_H.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS/h02da/H02A_R.da
household survey
/Users/zengsimon/Desktop/NYU/F

In [19]:
year = 2002
for x in output:
    filename = 'hrs_data_' + str(year) + '.pkl'
    x.to_pickle(filename)
    year += 2


In [20]:
test_df = pd.read_pickle('hrs_data_2002.pkl')

In [21]:
test_df

,HHID,PN,rmonth_survey,ryear_survey,age,live_nursing_facility,state_nursing_facility,year_born_x,born_us,state_born,country_born,education,highest_degree,race_latino_hispanic,race,num_children,religion,divorce_widow_status,marital_status,health_status,health_status_compared_to_prev,health_status_better_compared_to_prev_degree,health_status_worse_compared_to_prev_degree,high_blood_pressure,high_blood_pressure_compared_to_prev,diabetes,diabetes_compared_to_prev,cancer,cancer_compared_to_prev,lung_disease,lung_disease_compared_to_prev,heart_condition,heart_condition_compared_to_prev,stroke,heart_condition_compared_to_prev,psychiatric_emotional_problem,psychiatric_emotional_problem_compared_to_prev,arthritis,arthritis_compared_to_prev,pain,pain_degree,exercise,smoke,still_smoke,alcohol,num_drinks_days_per_week,num_drinks_per_day,shortness_breath,fatigue,cough,depressed_x,tired,appetite_loss,appetite_increase,memory_x,depressed_y,children_nearby,dependents,num_dependents,type_house,own_or_rent,property_value,year_property_acquired,vocab_level,friendliness,attentiveness,cooperation,tiredness,memory_y,hearing,employment_status,year_retired,state_live,gender,year_born_y,financial_expectation,leave_inheritance,income,debt_amount,wave
0,000003,010,4.0,2002.0,66.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,3.0,2.0,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,2.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,NaN,5.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,5.0,5.0,NaN,2.0,1.0,150000.0,NaN,2.0,3.0,3.0,1.0,2.0,3.0,1.0,1.0,NaN,9.0,1.0,1936.0,20.0,NaN,10000.0,NaN,6.0
1,000003,020,4.0,2002.0,63.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,3.0,1.0,2.0,NaN,5.0,NaN,5.0,NaN,1.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,NaN,1.0,2.0,1.0,1.0,5.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,5.0,5.0,NaN,2.0,1.0,150000.0,NaN,2.0,3.0,3.0,1.0,3.0,1.0,1.0,3.0,NaN,9.0,2.0,1938.0,80.0,NaN,10000.0,NaN,6.0
2,010001,010,4.0,2002.0,62.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,5.0,6.0,2.0,2.0,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,1.0,5.0,NaN,5.0,NaN,1.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,NaN,4.0,1.0,450000.0,NaN,3.0,3.0,3.0,1.0,3.0,1.0,1.0,5.0,1998.0,2.0,1.0,1939.0,50.0,0.0,NaN,NaN,6.0
3,010003,030,8.0,2002.0,46.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,5.0,5.0,1.0,1.0,1.0,NaN,1.0,3.0,5.0,NaN,5.0,NaN,1.0,1.0,1.0,NaN,1.0,5.0,1.0,2.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,5.0,NaN,4.0,1.0,10000.0,NaN,1.0,3.0,3.0,1.0,3.0,1.0,1.0,4.0,NaN,2.0,2.0,1956.0,0.0,NaN,NaN,NaN,6.0
4,010004,010,4.0,2002.0,62.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,2.0,3.0,NaN,4.0,1.0,2.0,5.0,NaN,1.0,1.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,3.0,1.0,2.0,1.0,NaN,5.0,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,5.0,NaN,2.0,1.0,200000.0,1987.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,4.0,NaN,2.0,1.0,1939.0,70.0,NaN,NaN,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97073,213468,010,9.0,2002.0,77.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,5.0,5.0,5.0,2.0,NaN,NaN,1.0,2.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,3.0,1.0,3.0,5.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,2.0,7.0,NaN,NaN,2.0,3.0,3.0,1.0,3.0,1.0,1.0,6.0,NaN,5.0,2.0,1925.0,NaN,NaN,NaN,NaN,6.0
97074,213471,010,6.0,2002.0,71.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,5.0,5.0,2.0,2.0,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,1.0,5.0,NaN,2.0,1.0,165000.0,NaN,2.0,3.0,3.0,2.0,3.0,1.0,1.0,1.0,NaN,5.0,2.0,1930.0,998.0,NaN,NaN,98.0,6.0
97075,213475,010,7.0,2002.0,77.0,5.0,NaN,NaN,NaN,

In [22]:
test_df2 = pd.read_pickle('hrs_data_2012.pkl')

In [23]:
test_df2[test_df2['HHID'] == "000003"]

,HHID,PN,rmonth_survey,ryear_survey,age,live_nursing_facility,state_nursing_facility,year_born_x,born_us,state_born,education,highest_degree,race_latino_hispanic,num_children,religion,divorce_widow_status,health_status,health_status_compared_to_prev,high_blood_pressure,high_blood_pressure_compared_to_prev,diabetes,diabetes_compared_to_prev,cancer,cancer_compared_to_prev,lung_disease,lung_disease_compared_to_prev,heart_condition,heart_condition_compared_to_prev,stroke,heart_condition_compared_to_prev,psychiatric_emotional_problem,psychiatric_emotional_problem_compared_to_prev,arthritis,arthritis_compared_to_prev,pain,pain_degree,smoke,still_smoke,alcohol,num_drinks_days_per_week,num_drinks_per_day,shortness_breath,fatigue,cough,depressed_x,tired,appetite_loss,appetite_increase,memory_x,depressed_y,children_nearby,dependents,num_dependents,type_house,own_or_rent,property_value,year_property_acquired,vocab_level,cooperation,tiredness,memory_y,hearing,employment_status,year_retired,state_live,gender,year_born_y,leave_inheritance,income,debt_amount,wave
0,000003,010,7.0,2012.0,76.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,2.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN,1.0,1.0,1.0,5.0,NaN,NaN,NaN,4.0,5.0,5.0,5.0,NaN,2.0,1.0,200000.0,NaN,2.0,3.0,1.0,NaN,NaN,5.0,NaN,9.0,1.0,1936.0,NaN,NaN,NaN,11.0
1,000003,020,7.0,2012.0,73.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,1.0,NaN,5.0,NaN,1.0,NaN,5.0,NaN,1.0,NaN,5.0,NaN,1.0,2.0,1.0,3.0,1.0,2.0,NaN,NaN,5.0,NaN,NaN,1.0,5.0,5.0,5.0,NaN,NaN,NaN,3.0,5.0,5.0,5.0,NaN,2.0,1.0,200000.0,NaN,2.0,3.0,1.0,NaN,NaN,5.0,NaN,9.0,2.0,1938.0,NaN,NaN,NaN,11.0


In [ ]:
output['hhidpn'] = output['HHID'] + output['PN']
output.groupby('wave').count()

,HHID,PN,rmonth_survey,ryear_survey,age,live_nursing_facility,state_nursing_facility,year_born_x,born_us,state_born,country_born,education,highest_degree,race_latino_hispanic,race,num_children,religion,divorce_widow_status,marital_status,health_status,health_status_compared_to_prev,health_status_better_compared_to_prev_degree,health_status_worse_compared_to_prev_degree,high_blood_pressure,high_blood_pressure_compared_to_prev,diabetes,diabetes_compared_to_prev,cancer,cancer_compared_to_prev,lung_disease,lung_disease_compared_to_prev,heart_condition,heart_condition_compared_to_prev,stroke,heart_condition_compared_to_prev,psychiatric_emotional_problem,psychiatric_emotional_problem_compared_to_prev,arthritis,arthritis_compared_to_prev,pain,pain_degree,exercise,smoke,still_smoke,alcohol,num_drinks_days_per_week,num_drinks_per_day,shortness_breath,fatigue,cough,depressed_x,tired,appetite_loss,appetite_increase,memory_x,depressed_y,children_nearby,dependents,num_dependents,type_house,own_or_rent,property_value,year_property_acquired,vocab_level,friendliness,attentiveness,cooperation,tiredness,memory_y,hearing,employment_status,year_retired,state_live,gender,year_born_y,financial_expectation,leave_inheritance,income,debt_amount,hhidpn
wave,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6.0,22267,22267,22267,22267,22191,22267,569,569,616,517,99,616,166,616,22214,616,614,8723,22266,22267,21651,2003,5915,22267,9398,22267,3072,22267,1450,22267,1609,22267,3690,22267,1585,22267,3074,22267,10984,22267,6864,22267,616,21995,22267,10534,7029,616,616,616,561,61,61,37,19986,19986,19820,21970,2205,21677,19656,15947,2744,22233,22027,22027,22027,22027,22027,22027,22253,2550,22163,22267,22267,19971,3346,8071,529,22267


## Determine labels dictionary now (death year for each HHID PN ID)

In [26]:
files = [
    "hrs_data_2002.pkl",
    "hrs_data_2004.pkl",
    "hrs_data_2006.pkl",
    "hrs_data_2008.pkl",
    "hrs_data_2010.pkl",
    "hrs_data_2012.pkl",
    "hrs_data_2014.pkl",
    "hrs_data_2016.pkl",
         ]

labels_dict = {}

df_2002 = pd.read_pickle('hrs_data_2002.pkl')
df_2004 = pd.read_pickle('hrs_data_2004.pkl')
df_2006 = pd.read_pickle('hrs_data_2006.pkl')
df_2008 = pd.read_pickle('hrs_data_2008.pkl')
df_2010 = pd.read_pickle('hrs_data_2010.pkl')
df_2012 = pd.read_pickle('hrs_data_2012.pkl')
df_2014 = pd.read_pickle('hrs_data_2014.pkl')
df_2016 = pd.read_pickle('hrs_data_2016.pkl')

dataframes = {
    2002: df_2002,
    2004: df_2004,
    2006: df_2006,
    2008: df_2008,
    2010: df_2010,
    2012: df_2012,
    2014: df_2014,
    2016: df_2016
}


In [28]:
sets_by_year = {year: set(zip(df['HHID'], df['PN'])) for year, df in dataframes.items()}

In [29]:
def find_earliest_missing_year(hhid, pn, start_year, sorted_years, sets_by_year):
    """
    Finds the earliest year where the given (HHID, PN) is missing, starting from a specific year.
    """
    for year in sorted_years:
        if year > start_year and (hhid, pn) not in sets_by_year[year]:
            return year
    return 0  # Return 0 if the individual is present in all years


In [31]:
results = []

# Get a sorted list of all years
all_years = sorted(dataframes.keys())

# Iterate through all DataFrames
for year in all_years:
    df = dataframes[year]
    # Add a column for the earliest missing year
    df['Earliest_Missing_Year'] = df.apply(
        lambda row: find_earliest_missing_year(row['HHID'], row['PN'], year, all_years, sets_by_year),
        axis=1
    )
    # Add year information and append to results
    df['Base_Year'] = year
    results.append(df[['HHID', 'PN', 'Base_Year', 'Earliest_Missing_Year']])

# Step 4: Combine results into a single DataFrame
final_results = pd.concat(results, ignore_index=True)

# Display the results
print(final_results)

            HHID   PN  Base_Year  Earliest_Missing_Year
0         000003  010       2002                   2014
1         000003  020       2002                   2016
2         010001  010       2002                      0
3         010003  030       2002                      0
4         010004  010       2002                   2012
...          ...  ...        ...                    ...
10124328  923525  010       2016                      0
10124329  923525  020       2016                      0
10124330  952836  010       2016                      0
10124331  958361  010       2016                      0
10124332  959738  010       2016                      0

[10124333 rows x 4 columns]


In [34]:
final_results['Earliest_Missing_Year'].value_counts()

Earliest_Missing_Year
0       6628356
2016    1137926
2014     881815
2010     709666
2008     498092
2012     198780
2006      63542
2004       6156
Name: count, dtype: int64

In [35]:
final_results.groupby(['Earliest_Missing_Year', 'Base_Year']).size().reset_index(name='counts')

,Earliest_Missing_Year,Base_Year,counts
0,0,2002,47722
1,0,2004,186336
2,0,2006,1317944
3,0,2008,875899
4,0,2010,859335
5,0,2012,1049895
6,0,2014,1046988
7,0,2016,1244237
8,2004,2002,6156
9,2006,2002,11344


In [36]:
final_results.to_pickle("hrs_labels.pkl")

## Trying to navigate through exit interviews

In [37]:
################### import packages ##########################################
import pandas as pd
import re
import numpy as np
import os
import copy 
import fnmatch
%load_ext line_profiler
pd.set_option('display.max_columns', None)

##############################################################################
####### choose variables to look for from codebooks ###########################
##############################################################################

# make sure these are all lowercase!!! #####################

# for HRS core files, 2002 - present
#SELECT VARIABLES OF INTEREST

variables_to_look_for = { 'a500':'rmonth_survey'
                         , 'a501':'ryear_survey'
                          , 'a123':'year_death'
                         }


# for HRS core files, pre 2002
# 2000 has g's, 1998 has f's, 1996 has e's, 1994 has w's, 1992 has v's
variables_to_look_for_pre_2002 = {
      'g775': 'rmonth_survey'
    , 'f704': 'rmonth_survey'
    , 'e391': 'rmonth_survey'
    , 'w120': 'rmonth_survey'
    , 'v128': 'rmonth_survey'
    , 'g774': 'ryear_survey'
    , 'f703': 'ryear_survey'
    , 'e393': 'ryear_survey'
    , 'w122': 'ryear_survey'
    , 'v127': 'ryear_survey'
    , 'g4996': 'rpr_lose_job'
    , 'f4583': 'rpr_lose_job'
    , 'e3788': 'rpr_lose_job'
    , 'w5801': 'rpr_lose_job' 
    , 'v3205': 'rpr_lose_job'    
}

var_dict = {
      1992: ''
    , 1994: ''
    , 1996: ''
    , 1998: ''
    , 2000: ''
    , 2002: 's'
    , 2004: 'j' # they skip i for some reason
    , 2006: 'k'
    , 2008: 'l'
    , 2010: 'm'
    , 2012: 'n'
    , 2014: 'o'
    , 2016: 'p' 
    , 2018: 'q'
    , 2020: 'r'
}

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [39]:
def sas_read_year(directory, variables_to_look_for, survey_year, variables_to_look_for_pre_2002):
    # HRS and CAMS assignment
    id_variables = ['hhid', 'pn']
    
    # HRS assignments
    if survey_year < 2001:
        variables_to_look_for = variables_to_look_for_pre_2002  
    # get directories given year and survey type
    directory_sas = directory + 'x' + str(survey_year)[2:4] + 'sas'
    directory_da = directory + 'x' + str(survey_year)[2:4] + 'da'
    # get survey specific variables
    if variables_to_look_for_pre_2002 != 'CAMS':
        survey_specific_variables = [var_dict[survey_year] + x for x in variables_to_look_for]

    # CAMS assignments
    if variables_to_look_for_pre_2002 == 'CAMS':
        # survey specific variables
        survey_specific_variables = [x + '_' + str(survey_year)[2:4] for x in variables_to_look_for]
        # CAMS directories
        # 2001 has a different directory name
        if survey_year == 2001:
            ext = 'cams' + str(survey_year)[2:4]
            ext_da = ext
        # 2009 has a different directory structure
        elif survey_year == 2009:
            ext = 'cams' + str(survey_year) + '/sas'
            ext_da = 'cams' + str(survey_year) + '/data'
        else:
            ext = 'cams' + str(survey_year)
            ext_da = ext
        directory_sas = directory + ext
        directory_da = directory + ext_da

    # get list of paths: pre 1996 and some cams dont have R in them
    if survey_year < 1996:
        path_list = sorted(list(set([directory_sas + '/' +  x.split('.')[0] 
                                for x in os.listdir(directory_sas) 
                                    if 'sas' in x.lower()])))
    else:
        path_list = sorted(list(set([directory_sas + '/' +  x.split('.')[0] 
                                for x in os.listdir(directory_sas) 
                                    if ('sas' in x.lower())
                                    and (
                                          ('r' in x[len(x.split('.')[0])-1].lower()) #respondent
                                        | ('h' in x[len(x.split('.')[0])-1].lower()) #household
                                        )
                                     
                            ]
                           )
                       )
                  )
        
    # from paths, get paths leading to files which have our desired variables
    # earlier than 2000, need to read all surveys because naming conventions and variables are unrelated
    # after 2000, only reads files with desired variables in them to save a lot of time
    path_short_list = path_list
    #print(path_short_list)
    '''
    if (survey_year > 2000) and (variables_to_look_for_pre_2002 != 'CAMS'):
        surveys = [x[0] for x in variables_to_look_for] + [x[0:2] for x in variables_to_look_for if 'lb' in x]
        path_short_list = [x for x in path_list if (('r' in x.split('/')[len(x.split('/'))-1].lower()  # get respondent
                                                        and any(y == x.split('/')[len(x.split('/'))-1].lower()[3:len(x.split('/')[len(x.split('/'))-1]) - 2] 
                                                                    for y in surveys) # get specific file
                                                    )  
                                                   ) 
                            or (('h' in x.split('/')[len(x.split('/'))-1].lower())  # get household
                                # take all surveys we are looking for, only keep if HH survey (this is the last list comprehension below)
                                and any(y == x.split('/')[len(x.split('/'))-1].lower()[3:len(x.split('/')[len(x.split('/'))-1]) - 2] 
                                                                    for y in [w for w in surveys if w in ['h', 'q', 'r', 'u']])
                                )
                           ]
    #
    (path_short_list)
    '''
    # rename variables to names chosen in input dictionary
    rename_dict = copy.deepcopy(variables_to_look_for)
    vars_list = [x for x in variables_to_look_for]
    for x in range(len(survey_specific_variables)):
        rename_dict[survey_specific_variables[x].upper()] = rename_dict.pop(vars_list[x])  
    print('looking for variables: ' + str(survey_specific_variables))
    
    # get proper file extensions
    # this maybe can get coded away if handle all the info in path and path short list better
    for i in range(len(path_short_list)):
        path = path_short_list[i]
        path_ind = len(path.split('/')) - 1
        if survey_year < 2001:
            ext_surv_dct = '/' + path.split('/')[path_ind] +  '.SAS' #in the year 2000, files are .SAS and not .sas
            ext_surv_da = '/' + path.split('/')[path_ind] + '.DA'
        else:
            ext_surv_dct = '/' + path.split('/')[path_ind] +  '.sas'
            ext_surv_da = '/' + path.split('/')[path_ind] + '.da'
        dct_file = directory_sas + ext_surv_dct
        filename = directory_da + ext_surv_da
        print(filename)
        # read the files
        temp_df = read_sas_fwf(dct_file, filename, survey_specific_variables)
        temp_df = temp_df.rename(columns = rename_dict)
        if i == 0:
            year_frame = temp_df
        else:
            try:
                year_frame = year_frame.merge(temp_df, on = ["HHID", "PN"], how = "outer")
            except:
                try:
                    ##### MERGE HOUSEHOLD DATA ON TO RESPONDENT LEVEL, GET DATASET UNIQUE AT THE HHID LEVEL ###############
                    # Lowest number is for the first person in the dataset with that HHID, so most likely oldest, or head of household(?)
                    ### We want the lowest QPN_FIN number for each HHID
                    ### If no QPN_FIN number for a HHID, select lowest QPN_FAM
                    ### IF no QPN_FIN or QPN_FAM, we want the lowest QPN_CS number
                    #temp_df = temp_df.sort_values(['HHID', 'PN_CS', 'PN_FIN', 'PN_FAM']).reset_index(drop = True)

                    # Keep top row for each HHID
                    #temp_df = temp_df.drop_duplicates(subset=['HHID'], keep = 'first')
                    
                    # THEN MERGE ONCE ROWS ARE UNIQUE TO HHID ###########
                    year_frame = year_frame.merge(temp_df, on = ["HHID"], how = "outer") # for households
                    print('household survey')
                except:
                    print('this is a pre-1996 household survey, so exclude') # pre-1996 some surveys wont have a PN

    # make the wave variable, 1992 was wave 1
    year_frame['wave'] = (survey_year - 1990) / 2
    return year_frame

In [42]:
cwd = os.getcwd()
directory = cwd + '/data/HRS_EXIT/'
years = [2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016
        ]
output = read_all_years(years, directory, variables_to_look_for, variables_to_look_for_pre_2002)

reading files for survey year: 2002
looking for variables: ['sa500', 'sa501', 'sa123']
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02A_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02B_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02C_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02D_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02E_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02G_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02IO_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02J_R.da
/Users/zengsimon/Desktop/NYU/Fall24/EthicalML/final_project/preprocess/data/HRS_EXIT/x02da/x02N_R.da
/Us

In [43]:
final_frame = pd.concat(output, ignore_index=True).drop_duplicates().reset_index(drop=True)


In [44]:
final_frame

,HHID,PN,rmonth_survey,ryear_survey,year_death,wave
0,000002,010,9.0,2002.0,2001.0,6.0
1,010460,010,11.0,2002.0,2002.0,6.0
2,010468,020,11.0,2002.0,2002.0,6.0
3,010565,030,8.0,2002.0,2001.0,6.0
4,010571,020,10.0,2002.0,2000.0,6.0
...,...,...,...,...,...,...
10548,918882,011,NaN,NaN,NaN,13.0
10549,919052,020,NaN,NaN,NaN,13.0
10550,919613,010,NaN,NaN,NaN,13.0
10551,920331,010,NaN,NaN,NaN,13.0


In [57]:
final_results.to_pickle("hrs_exits.pkl")

In [58]:
merged_df = df_2008.merge(final_frame[['HHID', 'PN', 'year_death']], on=['HHID', 'PN'], how='inner')


In [61]:
merged_df

,HHID,PN,rmonth_survey,ryear_survey,age,live_nursing_facility,state_nursing_facility,year_born_x,born_us,state_born,education,highest_degree,race_latino_hispanic,num_children,religion,divorce_widow_status,health_status,health_status_compared_to_prev,high_blood_pressure,high_blood_pressure_compared_to_prev,diabetes,diabetes_compared_to_prev,cancer,cancer_compared_to_prev,lung_disease,lung_disease_compared_to_prev,heart_condition,heart_condition_compared_to_prev,stroke,heart_condition_compared_to_prev,psychiatric_emotional_problem,psychiatric_emotional_problem_compared_to_prev,arthritis,arthritis_compared_to_prev,pain,pain_degree,smoke,still_smoke,alcohol,num_drinks_days_per_week,num_drinks_per_day,shortness_breath,fatigue,cough,depressed_x,tired,appetite_loss,appetite_increase,memory_x,depressed_y,children_nearby,dependents,num_dependents,type_house,own_or_rent,property_value,year_property_acquired,vocab_level,friendliness,cooperation,tiredness,memory_y,hearing,employment_status,year_retired,state_live,gender,year_born_y,leave_inheritance,income,debt_amount,wave,Earliest_Missing_Year,Base_Year,year_death
0,000003,010,2.0,2008.0,72.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,2.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,1.0,1.0,1.0,5.0,5.0,1.0,5.0,NaN,NaN,NaN,3.0,5.0,5.0,5.0,NaN,2.0,1.0,250000.0,NaN,2.0,5.0,3.0,1.0,NaN,NaN,5.0,NaN,9.0,1.0,1936.0,NaN,NaN,NaN,9.0,2014,2008,NaN
1,000003,020,2.0,2008.0,69.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,5.0,NaN,5.0,NaN,1.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,1.0,1.0,2.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,5.0,NaN,NaN,NaN,3.0,5.0,5.0,5.0,NaN,2.0,1.0,250000.0,NaN,2.0,5.0,3.0,1.0,NaN,NaN,5.0,NaN,9.0,2.0,1938.0,NaN,NaN,NaN,9.0,2016,2008,NaN
2,010001,010,9.0,2008.0,69.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,3.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,4.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,1.0,0.0,NaN,5.0,5.0,5.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,97.0,1.0,1000000.0,NaN,2.0,5.0,3.0,1.0,NaN,NaN,5.0,NaN,2.0,1.0,1939.0,0.0,NaN,NaN,9.0,0,2008,NaN
3,010003,030,3.0,2008.0,51.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,3.0,1.0,3.0,5.0,NaN,5.0,NaN,1.0,3.0,1.0,3.0,1.0,5.0,1.0,3.0,1.0,3.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,1.0,5.0,1.0,1.0,1.0,5.0,1.0,1.0,5.0,NaN,5.0,NaN,4.0,1.0,10000.0,NaN,2.0,5.0,3.0,1.0,NaN,NaN,4.0,NaN,2.0,2.0,1956.0,NaN,NaN,NaN,9.0,0,2008,NaN
4,010004,010,3.0,2008.0,68.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,1.0,2.0,5.0,NaN,1.0,8.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,1.0,2.0,1.0,1.0,NaN,5.0,1.0,3.0,3.0,1.0,1.0,5.0,1.0,NaN,NaN,NaN,4.0,5.0,5.0,5.0,NaN,2.0,1.0,260000.0,NaN,2.0,5.0,3.0,1.0,NaN,NaN,5.0,NaN,5.0,1.0,1939.0,NaN,NaN,NaN,9.0,2012,2008,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414917,502757,010,12.0,2008.0,58.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,2.0,1.0,1.0,1.0,5.0,NaN,5.0,NaN,5.0,NaN,1.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,5.0,5.0,NaN,NaN,5.0,5.0,5.0,5.0,NaN,NaN,NaN,3.0,5.0,1.0,5.0,NaN,4.0,2.0,NaN,NaN,2.0,5.0,3.0,1.0,NaN,NaN,1.0,NaN,2.0,2.0,1950.0,NaN,23093.0,NaN,9.0,2010,2008,NaN
1414918,502758,010,6.0,2008.0,56.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,1.0,1.0,NaN,1.0,2.0,1.0,NaN,5.0,NaN,5.0,NaN,1.0,5.0,5.0,NaN,1.0,NaN,1.0,2.0,NaN,5.0,1.0,0.0,NaN,5.0,5.0,1.0,5.0,NaN,NaN,NaN,3.0,5.0,NaN,5.0,NaN,3.0,2.0,NaN,NaN,2.0,5.0,1.0,1.0,NaN,NaN,1.0,NaN,9.0,1.0,1952.0,5.0,99999998.0,NaN,9.0,0,2008,NaN
1414919,502759,010,8.0,2008.0,59.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,1.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,5.0,1.0,3.0,3.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,2.0,5.0,1.0,5.0,NaN,2.0,1.0,380000.0,NaN,3.0,5.0,3.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1949.0,NaN,23000.0,NaN,9.0,0,2008,NaN
1414920,50